In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import VECM, select_order

In [2]:
# Set the range of prices and the number of rows
low = 1
high = 100
num_rows = 100

# Generate random prices
prices = np.random.uniform(low, high, size=(num_rows, 2))

# Convert prices to log prices
log_prices = np.log(prices)

# Create the DataFrame
df = pd.DataFrame(log_prices, columns=['Column1', 'Column2'])

In [3]:
df.head()

,Column1,Column2
0,3.966202,2.365655
1,4.470293,3.854192
2,4.557000,2.241115
3,3.443546,4.558134
4,2.660757,4.189898


In [10]:

def Bpdshare(x, override_lags=None, lag_max=None):
    if not isinstance(x, pd.DataFrame) or x.shape[1] != 2:
        raise ValueError("x must be a DataFrame with exactly two columns.")

    # Step 1: Determine the optimal number of lags
    if override_lags is None:
        model = VAR(x)
        lag_order_results = model.select_order(maxlags=lag_max)
        lags = lag_order_results.selected_orders['aic']
    else:
        lags = override_lags

    # Step 2: Estimate VECM
    vecm = VECM(x, k_ar_diff=lags, coint_rank=1, deterministic="co")
    vecm_fit = vecm.fit()

    # Step 3: Calculate var-covar matrix of residuals
    var_covar_matrix = np.cov(vecm_fit.resid.T)

    # Step 4: Calculate component share weights
    beta = vecm_fit.beta.T
    component_share = np.abs(beta) / np.sum(np.abs(beta))

    # Step 5: Calculate information shares
    is_original_ordering = np.zeros(2)
    is_reversed_ordering = np.zeros(2)
    for i in range(2):
        is_original_ordering[i] = var_covar_matrix[i, i] / np.sum(var_covar_matrix.diagonal())
        is_reversed_ordering[i] = var_covar_matrix[1-i, 1-i] / np.sum(var_covar_matrix.diagonal())

    return {
        'is.original.ordering': is_original_ordering,
        'is.reversed.ordering': is_reversed_ordering,
        'component.share': component_share,
        'var.covar.matrix': var_covar_matrix,
        'lags.used': lags,
    }

In [11]:
Bpdshare(df)

{'is.original.ordering': array([0.59440119, 0.40559881]),
 'is.reversed.ordering': array([0.40559881, 0.59440119]),
 'component.share': array([[0.00227174, 0.99772826]]),
 'var.covar.matrix': array([[1.13588688, 0.06627445],
        [0.06627445, 0.77508991]]),
 'lags.used': 0}